# TP 5-6 : Partie 2 : Problème avec précédences et ressources disjonctives

Initialization (à faire une seule fois)

In [1]:
#import Pkg; Pkg.add("Clp")

Récupération des données

In [2]:
MachinesParJob=[1 3 0; 1 2 3]
Duree=[6 7 0; 3 5 1]
bigM=sum(Duree)

22

Procédure d'application des tests de sondabilités TA, TO et TR pour le cas de la relaxation linéaire

In [3]:
function TestsSondabilite_relaxlin(model2, varsbin, BestTfin, Bestsol)
    TA, TO, TR = false, false, false
    if(termination_status(model2) == MOI.INFEASIBLE)#Test de faisabilite
        TA=true
        println("TA")
    elseif(objective_value(model2) >= BestTfin) #Test d'optimalite
        TO=true
        println("TO")
    elseif( prod(abs.([round.(v, digits=0) for v in value.(varsbin)]-value.(varsbin)) .<= fill(10^-5, size(varsbin))) 
        ) #Test de resolution
        TR=true
        println("TR")
        if (value(tfin) <= BestTfin)
            Bestsol = value.(t)
            BestTfin=value(tfin)
        end
    else
        println("non sondable")
    end
    TA, TO, TR, Bestsol, BestTfin
end

TestsSondabilite_relaxlin (generic function with 1 method)

Procédure de séparation et stratégie d'exploration permettant de se placer au prochain noeud à traiter

In [4]:

function SeparerNoeud_relaxlin(varsshouldbebinary, listvars, listvals)
    # le noeud est non-sondable. Appliquer le critère de séparation pour le séparer en sous-noeuds 
    # et choisir un noeud-fils le plus à gauche   
    
    #find a fractionnal variable
    i, var = 1, 0
    while((i <= length(varsshouldbebinary)) && (var==0))
        #if(varsshouldbebinary[i] ∉ listvars)
        if(abs(round(value(varsshouldbebinary[i]), digits=0) - value(varsshouldbebinary[i]) ) >= 10^-5)
            var=varsshouldbebinary[i]
        end
        i+=1
    end
    
    #=
    #find most fractionnal variable
    i, var, maxfrac = -1, 0, 0.0
    for i in 1:length(varsshouldbebinary)
        if(abs(round(value(varsshouldbebinary[i]), digits=0) - value(varsshouldbebinary[i]) ) >= maxfrac) 
            #if a variable is more fractinonal
            var=varsshouldbebinary[i]
            maxfrac=abs(round(value(varsshouldbebinary[i]), digits=0) - value(varsshouldbebinary[i]) )
            #println(i, " ", var, " ", maxfrac)
        end
    end
    =#
    

    set_lower_bound(var,1.0)
    set_upper_bound(var,1.0)

    push!(listvars,var) #stocker l'identite de la variable choisie pour la séparation
    push!(listvals,1.0) #stocker la branche choisie, identifiee par la valeur de la variable choisie
    listvars, listvals
end


function ExplorerAutreNoeud_relaxlin(listvars, listvals)
    #this node is sondable, go back to parent node then right child if possible
    
    stop=false
    #go back to parent node
    var=pop!(listvars)
    theval=pop!(listvals)
    set_lower_bound(var,0.0)
    set_upper_bound(var,1.0)

    #go to right child if possible, otherwise go back to parent
    while( (theval==0.0) && (length(listvars)>= 1))
        var=pop!(listvars)
        theval=pop!(listvals)
        set_lower_bound(var,0.0) 
        set_upper_bound(var,1.0)
    end
    if theval==1.0
        set_lower_bound(var,0.0)
        set_upper_bound(var,0.0)
        push!(listvars,var)
        push!(listvals,0.0)
    else
        println("\nFINISHED")
        stop=true
    end
    listvars, listvals, stop 
end

ExplorerAutreNoeud_relaxlin (generic function with 1 method)

Creation de la relaxation linéaire (= modèle associé au noeud 0)

In [5]:
# ROOT NODE
using JuMP
using Clp

model2 = Model(Clp.Optimizer) # set optimizer
set_optimizer_attribute(model2, "LogLevel", 0) #don't display anything during solve
set_optimizer_attribute(model2, "Algorithm", 4) #LP solver chosen is simplex

# define t variables
@variable(model2, t[i in 1:2, j in 1:3] >= 0)
@variable(model2, tfin)

# define objective function
@objective(model2, Min, tfin)

# define constraints: t_i(j+1) - t_ij  >= Duree[i,j], \forall i,j
@constraint(model2, t[1,2] - t[1,1] >= Duree[1,1] )
@constraint(model2, t[1,3] - t[1,2] >= Duree[1,2] )
@constraint(model2, t[2,2] - t[2,1] >= Duree[2,1] )
@constraint(model2, t[2,3] - t[2,2] >= Duree[2,2] )

#define constraints: tfin - t_ij >= Duree[i,j], \forall ij
@constraint(model2, tfin - t[1,3] >= Duree[1,3] )
@constraint(model2, tfin - t[2,3] >= Duree[2,3] )


# define x variables as CONTINUOUS (recall that it is not possible to define binary variables in Clp)
@variable(model2, 0 <= x_2_1__1_1 <= 1)
@variable(model2, 0 <= x_2_3__1_2 <= 1)
varsshouldbebinary=[x_2_1__1_1,x_2_3__1_2]


# define bigM constraints linking x and t variables
@constraint(model2, t[2,1] - t[1,1] >=  Duree[1,1] - bigM*x_2_1__1_1)
@constraint(model2, t[1,1] - t[2,1] >=  Duree[2,1] - bigM*(1-x_2_1__1_1))
@constraint(model2, t[2,3] - t[1,2] >=  Duree[1,2] - bigM*x_2_3__1_2)
@constraint(model2, t[1,2] - t[2,3] >=  Duree[2,3] - bigM*(1-x_2_3__1_2))


println(model2)

┌ Info: Precompiling JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1278
┌ Info: Precompiling Clp [e2554f3b-3117-50c0-817c-e040a3ddf72d]
└ @ Base loading.jl:1278


Min tfin
Subject to
 -t[1,1] + t[1,2] >= 6.0
 -t[1,2] + t[1,3] >= 7.0
 -t[2,1] + t[2,2] >= 3.0
 -t[2,2] + t[2,3] >= 5.0
 -t[1,3] + tfin >= 0.0
 -t[2,3] + tfin >= 1.0
 -t[1,1] + t[2,1] + 22 x_2_1__1_1 >= 6.0
 t[1,1] - t[2,1] - 22 x_2_1__1_1 >= -19.0
 -t[1,2] + t[2,3] + 22 x_2_3__1_2 >= 7.0
 t[1,2] - t[2,3] - 22 x_2_3__1_2 >= -21.0
 t[1,1] >= 0.0
 t[2,1] >= 0.0
 t[1,2] >= 0.0
 t[2,2] >= 0.0
 t[1,3] >= 0.0
 t[2,3] >= 0.0
 x_2_1__1_1 >= 0.0
 x_2_3__1_2 >= 0.0
 x_2_1__1_1 <= 1.0
 x_2_3__1_2 <= 1.0



Boucle principale : résoudre la relaxation linéaire, appliquer les tests de sondabilité, identifier le prochain noeud, répéter.

In [6]:

listvars=[]
listvals=[]

BestTfin=bigM
Bestsol=[]

current_node_number=0
stop = false

while(!stop)
    
    println("\nNode number ", current_node_number, ": \n-----\n", model2)

    print("Solve : start ... ")
    status = optimize!(model2)
    println("... end")

    println("\nSolution relax lin",); [print("\t", name(v),"=",value(v)) for v in all_variables(model2)]; 
    println(" "); println("\nSolution precedemment memorisee ", Bestsol, " avec date de fin ", BestTfin, "\n")

    TA, TO, TR, Bestsol, BestTfin = TestsSondabilite_relaxlin(model2, varsshouldbebinary, BestTfin, Bestsol)

    is_node_sondable = TA || TO || TR
    
    if(!is_node_sondable)
        listvars, listvals = SeparerNoeud_relaxlin(varsshouldbebinary, listvars, listvals)
    else
        listvars, listvals, stop = ExplorerAutreNoeud_relaxlin(listvars, listvals)
    end

    current_node_number = current_node_number + 1
end

println("\n******\n\nOptimal value = ", BestTfin, "\n\nOptimal t=", Bestsol)



Node number 0: 
-----
Min tfin
Subject to
 -t[1,1] + t[1,2] >= 6.0
 -t[1,2] + t[1,3] >= 7.0
 -t[2,1] + t[2,2] >= 3.0
 -t[2,2] + t[2,3] >= 5.0
 -t[1,3] + tfin >= 0.0
 -t[2,3] + tfin >= 1.0
 -t[1,1] + t[2,1] + 22 x_2_1__1_1 >= 6.0
 t[1,1] - t[2,1] - 22 x_2_1__1_1 >= -19.0
 -t[1,2] + t[2,3] + 22 x_2_3__1_2 >= 7.0
 t[1,2] - t[2,3] - 22 x_2_3__1_2 >= -21.0
 t[1,1] >= 0.0
 t[2,1] >= 0.0
 t[1,2] >= 0.0
 t[2,2] >= 0.0
 t[1,3] >= 0.0
 t[2,3] >= 0.0
 x_2_1__1_1 >= 0.0
 x_2_3__1_2 >= 0.0
 x_2_1__1_1 <= 1.0
 x_2_3__1_2 <= 1.0

Solve : start ... ... end

Solution relax lin
	t[1,1]=0.0	t[2,1]=0.0	t[1,2]=6.0	t[2,2]=3.0	t[1,3]=13.0	t[2,3]=8.0	tfin=13.0	x_2_1__1_1=0.27272727272727276	x_2_3__1_2=0.22727272727272732 

Solution precedemment memorisee Any[] avec date de fin 22

non sondable

Node number 1: 
-----
Min tfin
Subject to
 -t[1,1] + t[1,2] >= 6.0
 -t[1,2] + t[1,3] >= 7.0
 -t[2,1] + t[2,2] >= 3.0
 -t[2,2] + t[2,3] >= 5.0
 -t[1,3] + tfin >= 0.0
 -t[2,3] + tfin >= 1.0
 -t[1,1] + t[2,1] + 22 x_2_1_

In [7]:
println("\n******\n\nOptimal value = ", BestTfin, "\n\nOptimal t=", Bestsol)


******

Optimal value = 15.0

Optimal t=[0.0 6.0 15.0; 6.0 9.0 14.0]


In [ ]:
# Initialisation et création du type "graph"
sommet = Int64
poids = Float64
arete = Tuple{sommet,sommet,poids}

struct graph
    S::Array{sommet,1}
    A::Array{arete,1}
end
#ajoute un arc au graphe
function ajoute_arete(G::graph, a::arete)
    E=copy(G.A)
    push!(E, a)
    return graph(G.S,E)
end


# Renvoie tous les arcs de G qui arrivent vers le sommet s
function trouve_arcs(G::graph,s::sommet)
    S = G.S
    A = G.A
    nb_sommets = length(S)
    nb_aretes = length(A)
    L = []
    for a in 1:nb_aretes
        arete = A[a]
        if arete[2] == s
            append!(L,[arete])
        end
    end
    return L
end

function Bellman_Ford_max(G::graph,s::sommet,numerotfin::Int64)
    S = G.S
    A = G.A
    nb_sommets = length(S)
    nb_aretes = length(A)
    d = zeros(nb_sommets,nb_sommets)
    for i in 1:nb_sommets
        d[i,1] = typemin(Int64)
    end
    d[s,1] = 0
    for k in 2:nb_sommets
        for i in 1:nb_sommets
            dist = d[i,k-1]
            Arcs = trouve_arcs(G,i)
            for a in 1:length(Arcs)
                arc = Arcs[a]
                depart = arc[1]
                poids = arc[3]
                dist = max(dist,d[depart,k-1]+poids)
            end
            d[i,k] = dist
        end
    end
    return d[numerotfin,nb_sommets]
end


In [ ]:
function TestsSondabilite_relaxlin_gd(arc_disjonctifs_traites,disjonction_non_traitees, BestTfin, Bestsol,valeurNoeud)
    TA, TO, TR = false, false, false
    if (valeurNoeud == typemin(Int64))
        TA=true
        println("TA")
    elsif(valeurNoeud>=BestTfin)
        TO=true 
        println("TO")
    elseif(disjonction_non_traitees==[])
        TR=true
        ptrintln("TR")
        if (valeurNoeud<=BestTfin)
            BestTfin=valeurNoeud
            Bestsol=arc_disjonctifs_traites
    else 
        println("non sondable")
    TA, TO, TR, Bestsol, BestTfin
end

In [ ]:
function SeparerNoeud_relaxlin_gd(graphe, arc_disjonctifs_traites, valeurs_arcs_disjonctifs_traites,disjonctions_non_traitees)
    disjonction=pop!(disjonctions_non_traitees)
    arc=(disjonction[1],disjonction[2],disjonction[3][1])
    
end

In [1]:
function PSE_Graphe_Disjonctif(graphe, disjonctions,numerodepart,numerotfin)
    BestTfin=bigM
    Bestsol=[]
    current_node_number=0
    pile_graphe=[]
    push!(pile_graphe, graphe)
    stop = false
    disjonction_non_traitees=disjonctions
    arc_disjonctifs_traites=[]
    valeurs_arcs_disjonctifs_traites=[]
    while(!stop)
        println("\nNode number ", current_node_number, ": \n-----\n")
        valeurNoeud = Bellman_ford_max(graphe,numerodepart,numerotfin)
        println("\nSolution precedemment memorisee ", Bestsol, " avec date de fin ", BestTfin)
        TA, TO, TR, Bestsol, BestTfin = TestsSondabilite_relaxlin_gd(arc_disjonctifs_traites, disjonction_non_traitees, BestTfin, Bestsol,valeurNoeud)
        is_node_sondable = TA || TO || TR
        if(!is_node_sondable)
            arc_disjonctifs_traites, valeurs_arcs_disjonctifs_traites,disjonctions_non_traitees,graphe = SeparerNoeud_relaxlin_gd(graphe,arc_disjonctifs_traites, valeurs_arcs_disjonctifs_traites,disjonction_non_traitees)
        else
            arc_disjonctifs_traites, valeurs_arcs_disjonctifs_traites, stop = ExplorerAutreNoeud_relaxlin_gd(arc_disjonctifs_traites, valeurs_arcs_disjonctifs_traites)
        end

        current_node_number = current_node_number + 1
    end

    println("\n******\n\nOptimal value = ", BestTfin, "\n\nOptimal t=", Bestsol) 
end 

PSE_Graphe_Disjonctif (generic function with 1 method)